In [ ]:
# !pip install -q diffusers==0.25.0 wandb

In [1]:
# !pip install -q --upgrade diffusers huggingface_hub
!pip install e2cnn torchmetrics tqdm wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 5.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00:00:0100:01


In [ ]:
import os
os.kill(os.getpid(), 9)

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from diffusers import UNet2DConditionModel
import torch.nn.functional as F

2025-08-05 13:04:29.558748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754399069.583369     152 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754399069.591087     152 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!pip install -qU wandb 

In [5]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
CONFIG = {
    "epochs": 100,
    "batch_size": 16,
    "learning_rate": 1e-4,
    "img_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "dataset_path": "/kaggle/input/dataset-ayna/dataset",
    "project_name": "conditional-UNet-polygons",
    "run_name": f"run-img_size_{128}-bs_{16}",
}

print(f"Using device: {CONFIG['device']}")

Using device: cuda


In [9]:
train_json_path = os.path.join(CONFIG['dataset_path'], 'training', 'data.json')
with open(train_json_path, 'r') as f:
    train_data = json.load(f)

all_colors = sorted(list(set(item['colour'] for item in train_data)))
color_to_idx = {color: i for i, color in enumerate(all_colors)}
idx_to_color = {i: color for color, i in color_to_idx.items()}
NUM_COLORS = len(all_colors)

print(f"Found {NUM_COLORS} unique colors: {all_colors}")
CONFIG['num_colors'] = NUM_COLORS
CONFIG['color_map'] = color_to_idx

Found 8 unique colors: ['blue', 'cyan', 'green', 'magenta', 'orange', 'purple', 'red', 'yellow']


In [10]:
class PolygonDataset(Dataset):
    def __init__(self, dataset_path, split, color_map, transform=None):
        self.split_path = os.path.join(dataset_path, split)
        self.input_path = os.path.join(self.split_path, 'inputs')
        self.output_path = os.path.join(self.split_path, 'outputs')
        
        json_path = os.path.join(self.split_path, 'data.json')
        with open(json_path, 'r') as f:
            self.data = json.load(f)
            
        self.color_map = color_map
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        input_img_path = os.path.join(self.input_path, item['input_polygon'])
        output_img_path = os.path.join(self.output_path, item['output_image'])
        
        input_image = Image.open(input_img_path).convert("RGB")
        output_image = Image.open(output_img_path).convert("RGB")
        
        color_name = item['colour']
        color_idx = torch.tensor(self.color_map[color_name], dtype=torch.long)
        
        if self.transform:
            seed = np.random.randint(2147483647)
            torch.manual_seed(seed)
            input_image = self.transform(input_image)
            torch.manual_seed(seed)
            output_image = self.transform(output_image)
            
        return {
            'input_image': input_image,
            'color_idx': color_idx,
            'output_image': output_image
        }

In [11]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [12]:
data_transforms = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(), 
])

val_transforms = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.ToTensor(),
])

train_dataset = PolygonDataset(CONFIG['dataset_path'], 'training', color_to_idx, transform=data_transforms)
val_dataset = PolygonDataset(CONFIG['dataset_path'], 'validation', color_to_idx, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)


batch = next(iter(train_loader))
print("Input image batch shape:", batch['input_image'].shape)
print("Color index batch shape:", batch['color_idx'].shape)
print("Output image batch shape:", batch['output_image'].shape)

Input image batch shape: torch.Size([16, 3, 128, 128])
Color index batch shape: torch.Size([16])
Output image batch shape: torch.Size([16, 3, 128, 128])


In [13]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2,
                                    diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class ConditionalUNet(nn.Module):
    def __init__(self, n_channels, n_classes, num_colors, embedding_dim=32, bilinear=True):
        super(ConditionalUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.color_embedding = nn.Embedding(num_colors, embedding_dim)
        
        # Encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        
        self.embedding_proj = nn.Linear(embedding_dim, 1024 // factor)

        # Decoder
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, image, color_idx):
        # Encoder Path
        x1 = self.inc(image)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4) 

        c = self.color_embedding(color_idx) 
        c_proj = self.embedding_proj(c)
        c_proj = c_proj.unsqueeze(-1).unsqueeze(-1)
        x5 = x5 + c_proj
        
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        
        return torch.sigmoid(logits)


model = UNet2DConditionModel(
    sample_size=CONFIG['img_size'],          
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 256, 512), 
    down_block_types=(
        "DownBlock2D",
        "CrossAttnDownBlock2D", 
        "CrossAttnDownBlock2D",
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",
        "CrossAttnUpBlock2D",  
        "CrossAttnUpBlock2D",
        "UpBlock2D",
    ),
    cross_attention_dim=CONFIG['num_colors']
).to(CONFIG['device'])

print(f"Model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters.")      

Model has 69,047,811 trainable parameters.


In [14]:
!pip install -qU tqdm
!pip install -qU torchmetrics

In [15]:
from tqdm.notebook import tqdm
from torchmetrics import PeakSignalNoiseRatio
from torch.optim.lr_scheduler import CosineAnnealingLR

In [16]:
def train_model(model, config, train_loader, val_loader, color_map):
    wandb.init(project=config['project_name'], name=config['run_name'], config=config)
    
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    train_psnr = PeakSignalNoiseRatio().to(config['device'])
    val_psnr = PeakSignalNoiseRatio().to(config['device'])
    
    wandb.watch(model, criterion, log="all", log_freq=100)
    
    best_val_loss = float('inf')
    num_colors = config['num_colors']

    for epoch in range(config['epochs']):
        model.train()
        train_loss = 0.0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Training]")
        for batch in loop:
            input_imgs = batch['input_image'].to(config['device'])
            color_indices = batch['color_idx'].to(config['device'])
            target_imgs = batch['output_image'].to(config['device'])
            
            optimizer.zero_grad()
            
            color_cond = F.one_hot(color_indices, num_classes=num_colors).float().unsqueeze(1)
            outputs = model(sample=input_imgs, timestep=1, encoder_hidden_states=color_cond).sample
            
            loss = criterion(outputs, target_imgs)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * input_imgs.size(0)
            train_psnr.update(outputs, target_imgs)
            loop.set_postfix(loss=loss.item())
            
        train_loss /= len(train_loader.dataset)
        epoch_train_psnr = train_psnr.compute()
        train_psnr.reset() 


        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                input_imgs = batch['input_image'].to(config['device'])
                color_indices = batch['color_idx'].to(config['device'])
                target_imgs = batch['output_image'].to(config['device'])
                
                color_cond = F.one_hot(color_indices, num_classes=num_colors).float().unsqueeze(1)
                outputs = model(sample=input_imgs, timestep=1, encoder_hidden_states=color_cond).sample
                
                loss = criterion(outputs, target_imgs)
                val_loss += loss.item() * input_imgs.size(0)
                
                val_psnr.update(outputs, target_imgs)
        
        val_loss /= len(val_loader.dataset)
        epoch_val_psnr = val_psnr.compute()
        val_psnr.reset() 

        
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_accuracy_psnr": epoch_train_psnr,
            "val_accuracy_psnr": epoch_val_psnr,
        })
        
        print(
            f"Epoch {epoch+1}/{config['epochs']} | "
            f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
            f"Train Acc(PSNR): {epoch_train_psnr:.2f} | Val Acc(PSNR): {epoch_val_psnr:.2f}"
        )

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model.save_pretrained("best_model_diffusers")
            wandb.save("best_model_diffusers")

    wandb.finish()
    print("Training finished. Best model saved to 'best_model_diffusers/'")

In [17]:
train_model(model, CONFIG, train_loader, val_loader, color_to_idx)

/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(


Epoch 1/100 [Training]:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/100 | Train Loss: 0.5416 | Val Loss: 0.2393 | Train Acc(PSNR): 13.45 | Val Acc(PSNR): 17.86


Epoch 2/100 [Training]:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 2/100 | Train Loss: 0.1936 | Val Loss: 0.1372 | Train Acc(PSNR): 19.28 | Val Acc(PSNR): 21.02


...

Epoch 99/100 | Train Loss: 0.0192 | Val Loss: 0.0191 | Train Acc(PSNR): 36.54 | Val Acc(PSNR): 36.86


Epoch 100/100 [Training]:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 100/100 | Train Loss: 0.0192 | Val Loss: 0.0233 | Train Acc(PSNR): 36.65 | Val Acc(PSNR): 35.81


epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
train_accuracy_psnr,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇█▇▇█████
train_loss,█▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
val_accuracy_psnr,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▆▇▆▇▇▇▇█▇▇▇█▇▇█
val_loss,█▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
epoch,100
train_accuracy_psnr,36.65483
train_loss,0.01916
val_accuracy_psnr,35.80806
val_loss,0.02328


Training finished. Best model saved to 'best_model_diffusers/'
